In [27]:
!git clone https://github.com/pekatour/Charlie-DB.git
!mv ./Charlie-DB/dataset dataset
!rm -rf Charlie-DB/

Cloning into 'Charlie-DB'...
remote: Enumerating objects: 470, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 470 (delta 1), reused 8 (delta 1), pack-reused 462 (from 1)
Receiving objects: 100% (470/470), 226.84 MiB | 43.42 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Updating files: 100% (719/719), done.


In [28]:
!ls dataset

data.yaml  test  train	valid
